In [1]:
import pandas as pd
# File paths
train_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_train.csv"
test_file = r"C:\Users\BHARGAVI\Downloads\project_data\ghc_test.csv"

# Load CSV files into pandas DataFrames
df = pd.read_csv(train_file)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import RandomOverSampler  # Assuming you want to balance the dataset

# Assuming df is your DataFrame containing 'text' and 'label' columns

# Step 1: Drop rows with NaN in 'text' column
df = df.dropna(subset=['text'])

In [3]:
# Step 2: Split the data into X (features) and y (labels)
X = df['text']
y = df['label']

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=2000)  # Adjust max_features as needed

# Step 5: Fit and transform X_train (text data) using TfidfVectorizer
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Step 6: Transform X_test using the fitted TfidfVectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [4]:
# Step 7: Balance the training data (if needed)
ros = RandomOverSampler(random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train_tfidf, y_train)

# Step 8: Initialize and train the RandomForestClassifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_balanced, y_train_balanced)

# Step 9: Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Step 10: Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))


Accuracy: 0.8771526980482205
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.97      0.93      3800
           1       0.55      0.21      0.31       555

    accuracy                           0.88      4355
   macro avg       0.72      0.59      0.62      4355
weighted avg       0.85      0.88      0.85      4355

Confusion Matrix:
 [[3701   99]
 [ 436  119]]


RANDOM FOREST HYPERTUNING

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit the GridSearchCV instance on the balanced training data
grid_search.fit(X_train_balanced, y_train_balanced)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)
